In [3]:
import tatqa_utils
import pandas as pd

In [4]:
devdf = pd.read_json('dataset_raw/tatqa_dataset_dev.json')

In [5]:
import pandas as pd
import pyreadstat
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import utils
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import numpy as np
from progress.bar import Bar
%load_ext autoreload
%autoreload 2

In [6]:
import os
with open('dataset_raw/openai.api.key', 'r') as filek: 
    openai_key = filek.read()
os.environ["OPENAI_API_KEY"] =  openai_key 

In [7]:
from langchain.globals import set_llm_cache
from langchain_openai import ChatOpenAI
from langchain_community.cache import SQLiteCache

llm = ChatOpenAI(temperature=0)
set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [8]:
def gen_code_A(llm, question, table):
    prompt = f"Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return numeric results. Do not write explanation, just code.\nQuestion: {question} \n Table: {table}"
    return llm.invoke(prompt)
gen_code_A(llm, 'What is 1?', '[]')    

AIMessage(content='def run():\n    return 1', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 72, 'total_tokens': 79, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-119f123f-4923-41e1-b801-f856de8b359e-0', usage_metadata={'input_tokens': 72, 'output_tokens': 7, 'total_tokens': 79, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [13]:
def gen_code(llm, question, table):
    prompt = f"Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return numeric results. Do not write explanation, just code.\nQuestion: {question} \n Table: {table}"
    res = llm.invoke(prompt)
    code = res.content.replace('```python','').replace('```','')
    return code

def exec_code(code, table):  
    try: 
        loc = locals()   
        if not "run()" in code:
            exec(code + f"\nr = run({table})\n", globals(), loc)
        else: 
            exec(code + "\nr = run()\n", globals(), loc)
        return loc['r']
    except Exception as e:
            s = str(e)
            print('[Error] ' + s)
            return None
table = "[['', '', 'Years Ended September 30,', ''], ['', '2019', '2018', '2017'], ['Fixed Price', '$  1,452.4', '$  1,146.2', '$  1,036.9'], ['Other', '44.1', '56.7', '70.8'], ['Total sales', '$1,496.5', '$1,202.9', '$1,107.7']]"
code = gen_code(llm, 'What is the change in Other in 2019 from 2018?', table)
exec_code(code, table)

-12.600000000000001

In [23]:
from tqdm.notebook import tqdm as log_progress

res = []
for i, item in log_progress(devdf.iterrows()):
    table = item['table']['table']
    for q in item['questions']:        
        if q['answer_type'] == 'arithmetic' and 'table' in q['answer_from']: 
            code = gen_code(llm, q, table)
            pred = exec_code(code, table)
            #res.append({"table":table, "q":q, "pred":pred, "code": code})
            res.append(({"answer_type":q["answer_type"], "answer": q["answer"], 'scale': ''}, pred, '', q, code))


0it [00:00, ?it/s]

[Error] could not convert string to float: ''
178.66666666666666
[Error] '2019 €m' is not in list
[Error] invalid literal for int() with base 10: '(19911)'
73
[Error] could not convert string to float: 'Dividend yiel'
[Error] could not convert string to float: '1,372.3'
[Error] invalid literal for int() with base 10: ''
-14212
[Error] could not convert string to float: '1,073.3'
[Error] invalid literal for int() with base 10: '1,050'
[Error] invalid literal for int() with base 10: ''
[Error] '(' was never closed (<string>, line 3)
[Error] list index out of range
[Error] '(' was never closed (<string>, line 2)
[Error] invalid literal for int() with base 10: ''
[Error] invalid literal for int() with base 10: '(9)'
[Error] invalid literal for int() with base 10: '$(3990'
[Error] invalid literal for int() with base 10: '(1149550)'
2849050
[Error] '(' was never closed (<string>, line 5)
[Error] invalid syntax. Perhaps you forgot a comma? (<string>, line 3)
[Error] invalid literal for int() 

In [26]:
metrics = TaTQAEmAndF1()

for ans, pred, pred_scale, _,_ in res:
    metrics(ans, pred, pred_scale)
pred_em, pred_f1, scale_score, op_score = metrics.get_overall_metric(reset=True)
print( pred_em, pred_f1, scale_score)

0.6054131054131054 0.6054131054131054 0.8190883190883191


In [30]:
import pickle
with open('res_p1_noscale.pickle', 'wb') as f:
    pickle.dump(res,f)